<a href="https://colab.research.google.com/github/LLMUSER/on-device-tests/blob/main/CPU_QA_System(High_Inference_Time).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers huggingface-hub langchain auto_gptq chromadb sentence_transformers pypdf docx2txt llama-cpp-python ctransformers torch InstructorEmbedding langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.5 M

In [2]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [3]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# llama-2-7b-chat.Q4_K_M.gguf - LLM we are using

Mounted at /content/drive


In [5]:
!wget -P /content/drive/MyDrive/ "https://cdn-lfs.huggingface.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1694338619&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NDMzODYxOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg3Mzg1MzA0ZDRiZmE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=KXn%7EkxXdOve6jxWuksZqHkL0d1yK0bhVOaVkSr-S8bnmQDJDejHR3aEfOJLnt1uLCpNplGErUmbHffswDD9vpbW8v8jp5j%7EKjX%7EQDJw2FCIljEQDCvBXC6YmrgUhEROwzYyF3f3kbDgyohSolfAz278d89vZhtMRb8rY0f1GDGKyhZdxUUjm%7EXvU3%7EMDbvw3g5ROkLV%7Eh7nsT1airRPfRtSSUDTHt4hkOGGtOlemxIyLHSTPjkLArsK7FVrwts0FJj4EVE05oJPknVt3NNAyjmdqm0klu2oCXuuZVNC6Aq5huujDZnWqdCmoCxXBssNUT5LRYAt92QtbItI-TnIrxw__&Key-Pair-Id=KVTP0A1DKRTAX"
#If you already have downloaded in the drive then don't run this command.

The destination name is too long (991), reducing to 236
--2023-09-07 10:59:51--  https://cdn-lfs.huggingface.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1694338619&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NDMzODYxOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg3Mzg1MzA0ZDRiZmE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=KXn%7EkxXdOve6jxWuksZqHkL0d1yK0bhVOaVkSr-S8bnmQDJDejHR3aEfOJLnt1uLCpNplGErUmbHffswDD9vpbW8v8jp5j%7EKjX%7EQDJw2FCIljEQDCvBXC6YmrgUhEROwzYyF3f3kbDgyohSolfAz278d89vZhtMRb8rY

In [7]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader,UnstructuredMarkdownLoader,CSVLoader,UnstructuredHTMLLoader,UnstructuredExcelLoader,TextLoader,JSONLoader,Docx2txtLoader
pdf_loader = PyPDFLoader('India_new.pdf')
# readme_loader = DirectoryLoader('/content/data', glob="**/*.md",loader_cls=UnstructuredMarkdownLoader)
# txt_loader = DirectoryLoader('/content/data', glob="**/*.txt",loader_cls=TextLoader)
csv_loader = CSVLoader('test.csv')
# html_loader = DirectoryLoader('/content/data', glob="**/*.html",loader_cls=UnstructuredHTMLLoader)
# xlsx_loader=DirectoryLoader('/content/data', glob="**/*.xlsx",loader_cls=UnstructuredExcelLoader)
# json_loader = DirectoryLoader('/content/data',glob="**/*.json",loader_cls=JSONLoader)
docx_loader =  Docx2txtLoader('India_new_1.docx')

loaders=[pdf_loader , csv_loader,docx_loader]
documents=[]
for loader in loaders:
  documents.extend(loader.load())


In [17]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=50) #chunk overlap seems to work better
documents = text_splitter.split_documents(documents)
print(len(documents))

embeddings = HuggingFaceEmbeddings(model_name="msmarco-distilbert-base-tas-b")

113


In [ ]:
print(documents)

In [18]:
from langchain.vectorstores import Chroma
RETRIEVER = Chroma.from_documents(documents, embeddings,persist_directory="db")

In [23]:
ret = RETRIEVER.as_retriever()

In [13]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
def model_memory():
    # Adding history to the model.
    template = """Use the following pieces of context to answer the question at the end. If you don't know the answer,\
    just say that you don't know, don't try to make up an answer.

    {context}

    {history}
    Question: {question}
    Helpful Answer:"""

    prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
    memory = ConversationBufferMemory(input_key="question", memory_key="history")

    return prompt, memory

In [27]:
# Make sure the model path is correct for your system!
LLM = LlamaCpp(
    model_path="/content/drive/MyDrive/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=200,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True, # Verbose is required to pass to the callback manager
    n_ctx=2048
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [33]:
!pip install langchainhub

In [35]:
from langchain import hub
rag_prompt_llama = hub.pull("rlm/rag-prompt-llama")

In [36]:
from langchain.chains import RetrievalQA
prompt, memory = model_memory()

QA = RetrievalQA.from_chain_type(
        llm=LLM,
        chain_type="stuff",
        retriever=ret,
        return_source_documents=True,
        # chain_type_kwargs={"prompt": prompt, "memory": memory},
        chain_type_kwargs={"prompt": rag_prompt_llama}
    )

In [37]:
while True:
  prompt = input("Enter Your Query ? ")


    # If the user hits enter
  if prompt:
      # Then pass the prompt to the LLM
      # response = QA(prompt)
      QA({"query": prompt})
      # answer, docs = response["result"], response["source_documents"]
      # # ...and write it out to the screen
      # print(answer)
      # print("These are the documents where the response has been fetched from..- \n")
      # print(docs)

Enter Your Query ? who is justin ?


Llama.generate: prefix-match hit


KeyboardInterrupt: ignored

In [22]:

while True:
  user_input=input("Enter a Query ?")
  LLM(user_input)

Enter a Query ?tell me about QA retrieval system using langchain ?


QA retrieval systems are becoming increasingly popular due to their potential to improve the accuracy and efficiency of information retrieval tasks. One such approach is using language chaining, which involves combining multiple natural language processing (NLP) models to create a more robust and accurate QA retrieval system. In this answer, I will explain how a QA retrieval system using language chaining works and its benefits.
1. Introduction to Language Chaining:
Language chaining is a technique that combines the strengths of multiple NLP models to improve the accuracy and efficiency of information retrieval tasks. The basic idea is to use a series of NLP models, each with a specific function or task, to build a chain of predictions that can be used to retrieve relevant documents for a given query.
2. Components of a QA Retrieval System using Language Chaining:
A QA retrieval system using language chaining typicall

KeyboardInterrupt: ignored